### 1. Importing Required Libraries for YOLO Model and for Training

In [2]:
!pip install ultralytics torch torchvision


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [1]:
from ultralytics import YOLO
import torch